# MISTral RAG Thing

### imports

In [5]:
import os
import torch
import torchvision
import torchaudio
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

home_dir = os.getenv("HOME")

model_name = f'{home_dir}/ext-gits/Mistral-7B-Instruct-v0.3'

tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

try:
    # Attempt to use 8-bit quantization
    quantization_config = BitsAndBytesConfig(load_in_8bit=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        device_map={"": device},
        trust_remote_code=True
    )
    print("Model loaded with 8-bit quantization.")
except Exception as e:
    print(f"8-bit quantization failed: {e}")
    print("Falling back to Float16 precision.")
    # Load the model with Float16 precision
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map={"": device},
        trust_remote_code=True
    )



/Users/crashy/ext-gits/Mistral-7B-Instruct-v0.3
Using device: mps
8-bit quantization failed: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`
Falling back to Float16 precision.


Loading checkpoint shards: 100%|██████████| 3/3 [00:19<00:00,  6.43s/it]


In [8]:

prompt = "Once upon a time"

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(device)

# Generate response
with torch.no_grad():
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=50,
        num_return_sequences=1,
        do_sample=True,
        top_p=0.95,
        top_k=50,
        pad_token_id=tokenizer.pad_token_id
    )

# Decode and print the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)

Once upon a time in a land far, far away, there was a little girl named Ella. Ella was a quiet, meek girl who felt uncomfortable and self-conscious in her own skin. She had big, drooping ears
